In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [55]:
# Read Excel file and save as DataFrame

df = pd.read_excel('./Datasets/tobacco_data.xlsx')
df.columns = df.iloc[0]
df = df[1:]
df.head()

,Intervention_descriptor,tax_increase,outlet_reduction,dec_smoking_prevalence,dec_tobacco_supply,dec_smoking_uptake,age,gender,ethnicity,discount_rate,evidence_strength,qalys_pc,hs_costs_pc
1,Combined tobacco endgame strategy (tobacco-fre...,10,90,7,0,0,0-14,Male,non-Māori,0,NaN,40.865526,-1284765.096725
2,Combined tobacco endgame strategy (tobacco-fre...,10,90,7,0,0,15-24,Male,non-Māori,0,NaN,41.708939,-1270055.987675
3,Combined tobacco endgame strategy (tobacco-fre...,10,90,7,0,0,25-44,Male,non-Māori,0,NaN,13.282615,-318700.524314
4,Combined tobacco endgame strategy (tobacco-fre...,10,90,1,0,0,45-64,Male,non-Māori,0,NaN,7.222291,-119003.652181
5,Combined tobacco endgame strategy (tobacco-fre...,10,90,0.5,0,0,65+,Male,non-Māori,0,NaN,1.111505,-9656.694651


In [56]:
# Transform data

# Map age group to integer
age_group_mapping = {
    '0-14': 0,
    '15-24': 1,
    '25-44': 2,
    '45-64': 3,
    '65+': 4
}

# Map gender to integer
gender_mapping = {
    'Male': 0,
    'Female': 1
}

# Map ethnicity to integer
ethnicity_mapping = {
    'Māori': 0,
    'non-Māori': 1
}

# Apply the mapping to the 'Age_Group' column
df['age_group'] = df['age'].map(age_group_mapping)
df['gender_idx'] = df['gender'].map(gender_mapping)
df['ethnicity_idx'] = df['ethnicity'].map(ethnicity_mapping)

# Convert the specified columns to floats
df[['tax_increase', 'outlet_reduction', 'dec_smoking_prevalence', 
    'dec_tobacco_supply', 'dec_smoking_uptake']] = df[['tax_increase', 'outlet_reduction', 
    'dec_smoking_prevalence', 'dec_tobacco_supply', 'dec_smoking_uptake']].apply(pd.to_numeric, errors='coerce').astype('float')

# Display the updated DataFrame
df.head()

,Intervention_descriptor,tax_increase,outlet_reduction,dec_smoking_prevalence,dec_tobacco_supply,dec_smoking_uptake,age,gender,ethnicity,discount_rate,evidence_strength,qalys_pc,hs_costs_pc,age_group,gender_idx,ethnicity_idx
1,Combined tobacco endgame strategy (tobacco-fre...,10.0,90.0,7.0,0.0,0.0,0-14,Male,non-Māori,0,NaN,40.865526,-1284765.096725,0.0,0.0,1.0
2,Combined tobacco endgame strategy (tobacco-fre...,10.0,90.0,7.0,0.0,0.0,15-24,Male,non-Māori,0,NaN,41.708939,-1270055.987675,1.0,0.0,1.0
3,Combined tobacco endgame strategy (tobacco-fre...,10.0,90.0,7.0,0.0,0.0,25-44,Male,non-Māori,0,NaN,13.282615,-318700.524314,2.0,0.0,1.0
4,Combined tobacco endgame strategy (tobacco-fre...,10.0,90.0,1.0,0.0,0.0,45-64,Male,non-Māori,0,NaN,7.222291,-119003.652181,3.0,0.0,1.0
5,Combined tobacco endgame strategy (tobacco-fre...,10.0,90.0,0.5,0.0,0.0,65+,Male,non-Māori,0,NaN,1.111505,-9656.694651,4.0,0.0,1.0


In [85]:
# Duplicate the DataFrame 3 times
df_duplicated = pd.concat([df]*5, ignore_index=True)

numeric_columns = ['tax_increase', 'outlet_reduction', 'dec_smoking_prevalence', 
                   'dec_tobacco_supply', 'dec_smoking_uptake', 'qalys_pc', 'hs_costs_pc']

# Set the seed for reproducibility
np.random.seed(42)

# Add Gaussian noise to the numeric columns with a standard deviation of 0.1
for col in numeric_columns:
    df_duplicated[col] += np.random.normal(0, 0.1, df_duplicated[col].shape)

# Define dependent and independent variables
X = np.array(df_duplicated[['tax_increase', 'outlet_reduction', 'dec_smoking_prevalence', 
                            'dec_tobacco_supply', 'dec_smoking_uptake', 'age_group', 
                            'gender_idx', 'ethnicity_idx']])

y = np.array(df_duplicated[['qalys_pc', 'hs_costs_pc']])

In [86]:
# Define Mean Absolute Percentage Error

def mape(y_actual, y_predicted):
    # Convert inputs to numpy arrays for element-wise operations
    y_actual = np.array(y_actual)
    y_predicted = np.array(y_predicted)
    
    # Ensure y_actual and y_predicted have the same length
    assert len(y_actual) == len(y_predicted)
    
    # Avoid division by zero by excluding zero values in y_actual
    non_zero_indices = y_actual != 0
    
    # Calculate MAPE only on non-zero actual values
    error = np.abs((y_actual[non_zero_indices] - y_predicted[non_zero_indices]) / y_actual[non_zero_indices])
    
    # Return the mean error
    return np.mean(error)

In [87]:
# Split the dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# List of different values of n_estimators to test
n_estimators_list = list(range(10, 100, 10))
max_depths_list = list(range(5, 25, 5))

# 2D Array to store MAPE for different n_estimators and values
mape_qalys_arr = np.zeros((len(n_estimators_list), len(max_depths_list)))
mape_costs_arr = np.zeros((len(n_estimators_list), len(max_depths_list)))

# Loop over the values of n_estimators and max_depth
for i in range(len(n_estimators_list)):
    for j in range(len(max_depths_list)):
        # Initialize the RandomForestRegressor with the current n_estimators and max_depth
        rf1 = RandomForestRegressor(n_estimators=n_estimators_list[i], max_depth=max_depths_list[j], random_state=42)
        rf2 = RandomForestRegressor(n_estimators=n_estimators_list[i], max_depth=max_depths_list[j], random_state=42)
        
        # Train the model on the training data for QALYs and costs
        rf1.fit(X_train, y_train[:, 0])
        rf2.fit(X_train, y_train[:, 1])
                                    
        # Make predictions on the validation set
        y_pred_qaly = rf1.predict(X_val)
        y_pred_hse = rf2.predict(X_val)

        mape_qaly = mape(y_val[:, 0], y_pred_qaly)
        mape_cost = mape(y_val[:, 1], y_pred_hse)
        
        # Store the relative MAPE in the arrays
        mape_qalys_arr[i][j] = mape_qaly
        mape_costs_arr[i][j] = mape_cost

In [88]:
# Retrieve best hyperparameters

qaly_min_index = np.unravel_index(np.argmin(rmse_qalys_arr), rmse_qalys_arr.shape)
qaly_n_estimators = n_estimators_list[qaly_min_index[0]]
qaly_max_depth = max_depths_list[qaly_min_index[1]]

hse_min_index = np.unravel_index(np.argmin(rmse_costs_arr), rmse_costs_arr.shape)
hse_n_estimators = n_estimators_list[hse_min_index[0]]
hse_max_depth = max_depths_list[hse_min_index[1]]

In [89]:
# Testing using best n_estimators and max_depth
rf1_best = RandomForestRegressor(n_estimators=qaly_n_estimators, max_depth = qaly_max_depth, random_state=42)
rf1_best.fit(X_train, y_train[:,0])
rf2_best = RandomForestRegressor(n_estimators=hse_n_estimators, max_depth = hse_max_depth, random_state=42)
rf2_best.fit(X_train, y_train[:,1])

# Make predictions on the test set
qaly_pred = rf1_best.predict(X_test)
cost_pred = rf2_best.predict(X_test)

# Compute MAPE for each output
mape_qaly_test = mape(y_test[:, 0], qaly_pred)
mape_cost_test = mape(y_test[:, 1], cost_pred)

print(f'n_estimators for QALYs = {qaly_n_estimators}')
print(f'max_depth for QALYs = {qaly_max_depth}')
print(f'MAPE for QALYs: {mape_qaly_test}')
print("----------------------------------")
print(f'n_estimators for Health System Cost = {hse_n_estimators}')
print(f'max_depth for Health System Cost = {hse_max_depth}')
print(f'MAPE for Health System Cost: {mape_cost_test}')

n_estimators for QALYs = 50
max_depth for QALYs = 15
MAPE for QALYs: 0.8790710615124909
----------------------------------
n_estimators for Health System Cost = 50
max_depth for Health System Cost = 20
MAPE for Health System Cost: 1.4829325064545524
